In [22]:
#webscraping here

import pandas as pd
import praw

reddit = praw.Reddit(client_id="69xO_MRhRBIQGprtVEZIbA",
    client_secret="GgPpkwrp_BPNduxYQ-dOcxfmrykfhQ",
    user_agent="test")


subreddit = reddit.subreddit("scu")

posts_dict = {"Body": [], "ID": [], "Ups": []}

for post in subreddit.hot(limit=1000):
    # Title and body of each post
    title = post.title
    text = post.selftext
    body = title + "\n" + text
    posts_dict['Body'].append(body)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Ups"].append(post.score)

top_posts = pd.DataFrame(posts_dict)
top_posts.to_csv("Top Posts.csv", index=True)
display(top_posts)



,Body,ID,Ups
0,Recruitments for Studies/Job Opportunities & O...,oouus2,17
1,Advice Paper for Incoming COEN/CS Students\nHi...,xkfy3u,46
2,Robotics and Automation in SCU\nI am from Asia...,1dbiqsj,1
3,"CRE QUESTION\nHello,\n\nI’ve took AP Calculus ...",1dbe0b2,2
4,Entire Townhome Available for Sublet//1st Quar...,1db2tb3,1
...,...,...,...
968,Freshman dorms\nHow hard is it to get into Fin...,kxpxqv,4
969,(Open to all Bay Area Residents) 1 Min Survey ...,kwq8y5,4
970,Hackathon Opportunity For SCU Students (All Ma...,ku4vj1,11
971,"Calling all engineering, computer science, and...",ks6uiv,3


In [23]:

id_list = top_posts["ID"].tolist()
#id_list = ["1d5yi68", "1cogl0"]
comments = pd.DataFrame()
comment_list = []
for i in id_list:                      #indicates start of the subission id processing
    submission = reddit.submission(i)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if(comment.author == "AutoModerator"):
            continue
        else:
            comment_list.append(comment.__dict__)

comments_df = pd.DataFrame(comment_list)

cleaned_df = comments_df.filter(['_submission','id','author', 'body', 'ups'], axis=1)

#cleaned_df.to_csv(f'comments.csv')

display(cleaned_df)

,_submission,id,author,body,ups
0,1cp9tqh,l3jhjuz,Altruistic-Fudge-522,Actually very good maybe retake SAT that's the...,4
1,1cp9tqh,l3jqth6,lucatitoq,You’ll probably get in. Although my GPA was sl...,4
2,1cp9tqh,l3kus0d,holiztic,ED1 with your junior year GPA trend and ECs so...,2
3,1cp9tqh,l3mbirn,travs-scott,I think you’d get in EA,1
4,1cp9tqh,l3mvftz,IceonFir3,"1460, 3.96/4.23, 3 APs at the time of applicat...",1
...,...,...,...,...,...
222,osylwk,h8yr8qb,Speedbird216,What I did is I went to the Cellar market and ...,1
223,kxpxqv,gjbwypo,iEshkker,Graham is pretty hard to get into mostly becau...,3
224,kxpxqv,gjbrxzg,Hookedonnetflix,Not hard,2
225,kxpxqv,gjenjk9,Bronco687,Swig fall 2021 bouta be crazy as fuck,1


In [23]:
import nltk
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

dorm_names = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin-walsh', 'casa italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university villas', 'nobili']

# Function to extract dorm names from comments
def dorm_appears(comment, dorm):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)
    return dorm in tokens

# Check if at least one dorm is found in the comment
def any_dorm_found(row):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_") 
        if row[dorm_column]:
            return True
    return False

for dorm in dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
    #display(cleaned_df['body'])
    top_posts[dorm_column] = top_posts['Body'].apply(dorm_appears, dorm=dorm)

# Remove rows where no dorms  found
top_posts = top_posts[top_posts.apply(any_dorm_found, axis=1)]

display(top_posts)

,Body,ID,Ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili
38,What are my chances?\nHey everyone!\n\nI'm cur...,1cp9tqh,3,False,False,False,False,False,False,False,False,False,True,False,False,False,False
49,Graham/Finn\nDo incoming freshman have a chanc...,1cmhmzn,3,True,True,False,False,False,False,False,False,False,False,False,False,False,False
67,"Transfer Housing Information\nHello, I'm a tra...",1chxhvo,3,False,True,False,False,False,False,False,False,False,False,False,False,False,False
76,Orientation Sessions are open for registration...,1cd3a17,6,False,False,False,False,False,False,False,True,False,False,False,False,False,False
80,Dunne bathroom updated?\nI am about to submit ...,1camxec,1,False,False,False,False,False,False,False,False,False,False,False,True,False,False
101,"Santa Clara University or UC Santa Cruz, Which...",1c22i9j,2,False,False,False,False,False,False,False,False,True,False,False,False,False,False
114,Complaining about Swig & Swig Residents until ...,1byptue,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False
156,Housing at SCU\nI was wondering how housing wo...,1bmxuwd,2,True,False,False,False,False,False,False,False,False,False,True,False,False,False
183,If you’re a first year honors student are you ...,1bh9dfi,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False
186,"Rank the dorms\nPlease rank\n\nCasa, campesi, ...",1bf62mi,7,False,False,True,False,False,False,True,False,False,False,True,False,False,False


In [24]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess_and_tokenize(text):
    # Clean the text as previously defined
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    cleaned_text = " ".join(new_text)
    
    # Tokenize and ensure consistent input size with padding and truncation
    return tokenizer(cleaned_text, padding="max_length", max_length=128, truncation=True, return_tensors='pt')
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

def sent_analyze(text):
    try:
        encoded_input = preprocess_and_tokenize(text)
        output = model(**encoded_input)
        scores = output.logits.detach().numpy()
        scores = softmax(scores, axis=1)
        return scores[0]  # Return the first batch as you're processing texts one by one
    except Exception as e:
        print(f"Error processing text: {text}")
        print(e)
        return [0, 0, 0]  # Return a default value in case of error


"""x = sent_analyze("Each dorm has its own personality, I was in Sobrato when I should have been in Swig/Dunn.")
print(x[0], "," , x[1], "," , x[2]) """

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

'x = sent_analyze("Each dorm has its own personality, I was in Sobrato when I should have been in Swig/Dunn.")\nprint(x[0], "," , x[1], "," , x[2]) '

In [25]:

import pandas as pd
from scipy.special import softmax

# Assuming the rest of your setup (imports and function definitions like sent_analyze) is correct and above this code block

# Apply the sentiment analysis function to each row in the 'Body' column
top_posts["mcwalsh"] = top_posts["mclaughlin_walsh"] | top_posts["walsh"]
top_posts["campisi_camp"] = top_posts["campisi"] | top_posts["camp"]
top_posts["sanfilippo_flip"] = top_posts["sanfilippo"] | top_posts["flip"]
top_posts.drop(columns=["walsh", "camp", "flip", "campisi", "sanfilippo", "mclaughlin_walsh"], inplace=True)



top_posts['sentiment'] = top_posts['Body'].apply(sent_analyze)

# Create new columns for negative, neutral, and positive scores by extracting the respective indices from the sentiment list
top_posts['neg'] = top_posts['sentiment'].apply(lambda x: x[0])
top_posts['neu'] = top_posts['sentiment'].apply(lambda x: x[1])
top_posts['pos'] = top_posts['sentiment'].apply(lambda x: x[2])

# Optionally, you can drop the 'sentiment' column if it's no longer needed
top_posts.drop('sentiment', axis=1, inplace=True)

# View the DataFrame to confirm the new columns are added correctly
display(top_posts)

#turn comments_cleaned.csv into a df then do the same as above for new df
comments_cleaned_df = pd.read_csv('comments_cleaned.csv')
comments_cleaned_df["mcwalsh"] = comments_cleaned_df["mclaughlin_walsh"] | comments_cleaned_df["walsh"]
comments_cleaned_df["campisi_camp"] = comments_cleaned_df["campisi"] | comments_cleaned_df["camp"]
comments_cleaned_df["sanfilippo_flip"] = comments_cleaned_df["sanfilippo"] | comments_cleaned_df["flip"]
comments_cleaned_df.drop(columns=["walsh", "camp", "flip", "campisi", "sanfilippo", "mclaughlin_walsh"], inplace=True)

comments_cleaned_df['sentiment'] = comments_cleaned_df['body'].apply(sent_analyze)
comments_cleaned_df['neg'] = comments_cleaned_df['sentiment'].apply(lambda x: x[0])
comments_cleaned_df['neu'] = comments_cleaned_df['sentiment'].apply(lambda x: x[1])
comments_cleaned_df['pos'] = comments_cleaned_df['sentiment'].apply(lambda x: x[2])
comments_cleaned_df.drop('sentiment', axis=1, inplace=True)
display(comments_cleaned_df)



,Body,ID,Ups,graham,finn,casa_italiana,sobrato,swig,casa,university_villas,nobili,mcwalsh,campisi_camp,sanfilippo_flip,neg,neu,pos
38,What are my chances?\nHey everyone!\n\nI'm cur...,1cp9tqh,3,False,False,False,False,False,False,False,False,False,True,False,0.010929,0.473358,0.515713
49,Graham/Finn\nDo incoming freshman have a chanc...,1cmhmzn,3,True,True,False,False,False,False,False,False,False,False,False,0.014901,0.957815,0.027284
67,"Transfer Housing Information\nHello, I'm a tra...",1chxhvo,3,False,True,False,False,False,False,False,False,False,False,False,0.018936,0.893320,0.087744
76,Orientation Sessions are open for registration...,1cd3a17,6,False,False,False,False,True,False,False,False,False,False,False,0.006647,0.892695,0.100658
80,Dunne bathroom updated?\nI am about to submit ...,1camxec,1,False,False,False,False,False,False,False,False,True,False,False,0.090354,0.853742,0.055904
101,"Santa Clara University or UC Santa Cruz, Which...",1c22i9j,2,False,False,False,False,False,False,False,False,False,False,True,0.012864,0.678406,0.308730
114,Complaining about Swig & Swig Residents until ...,1byptue,2,False,False,False,False,True,False,False,False,False,False,False,0.382792,0.564780,0.052428
156,Housing at SCU\nI was wondering how housing wo...,1bmxuwd,2,True,False,False,False,False,True,False,False,False,False,False,0.063373,0.844904,0.091723
183,If you’re a first year honors student are you ...,1bh9dfi,2,False,False,False,False,True,False,False,False,False,False,False,0.046505,0.909442,0.044052
186,"Rank the dorms\nPlease rank\n\nCasa, campesi, ...",1bf62mi,7,False,False,False,True,False,True,False,False,False,False,True,0.025741,0.808154,0.166104


,Unnamed: 0,_submission,id,author,body,ups,graham,finn,casa_italiana,sobrato,swig,casa,university_villas,nobili,mcwalsh,campisi_camp,sanfilippo_flip,neg,neu,pos
0,48,1d5yi68,l6ostw4,Ch8ngus,Mo clue about being trans but graham and Finn ...,6,True,True,False,False,False,False,False,False,False,False,False,0.011656,0.081077,0.907267
1,49,1d5yi68,l6oy0fr,5torm,Swig is “sustainability” and Casa is “Italian”).,4,False,False,False,False,True,True,False,False,False,False,False,0.057658,0.790006,0.152336
2,51,1d5yi68,l6seh8v,Affectionate_Dog_600,"Swig, you will find it very inclusive there",-1,False,False,False,False,True,False,False,False,False,False,False,0.005366,0.099713,0.894921
3,126,1cqr2oq,l576uig,Skicrazy85,"Each dorm has its own personality, I was in So...",2,False,False,False,True,True,False,False,False,False,False,False,0.131188,0.789410,0.079402
4,163,1cm4xod,l36tyz2,Magellan_8888,It will be quite easy meeting people. I’m curr...,4,False,False,False,False,False,True,False,False,False,False,False,0.004078,0.137564,0.858359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,3857,m8i6px,grojo66,SCUSKU,Congrats on the scholarship! That in and of it...,3,False,False,False,False,True,False,False,False,False,False,False,0.003490,0.014245,0.982264
253,4014,kxpxqv,gjbwypo,iEshkker,Graham is pretty hard to get into mostly becau...,3,True,False,False,False,False,False,False,False,False,False,False,0.046387,0.396804,0.556810
254,4016,kxpxqv,gjenjk9,Bronco687,Swig fall 2021 bouta be crazy as fuck,1,False,False,False,False,True,False,False,False,False,False,False,0.117957,0.396551,0.485491
255,4017,kxpxqv,gjdsg09,waffleseeds,Well this is my freshman year. I got into Dunn...,1,True,False,False,False,False,False,False,False,False,False,False,0.340526,0.617597,0.041878


In [34]:


#go through top posts and cleaned_comments ad avg the neg, neu, pos scores for each dorm
combined_dorm_names = dorm_names = ['graham', 'finn', 'sanfilippo_flip', 'campisi_camp', 'mcwalsh', 'casa italiana', 'sobrato', 'swig', 'casa', 'university villas', 'nobili']
dorm_sentiment = {}
for dorm in combined_dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")
    dorm_posts = top_posts[top_posts[dorm_column]]
    dorm_comments = comments_cleaned_df[comments_cleaned_df[dorm_column]]
    if(len(dorm_posts) + len(dorm_comments) == 0):
            continue
    else:
        dorm_sentiment[dorm] = {
            "neg": [(dorm_posts["neg"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["neg"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))],
            "neu": [(dorm_posts["neu"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["neu"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))],
            "pos": [(dorm_posts["pos"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["pos"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))]
        }


display(dorm_sentiment)

#print ranking based on sentiment analysis: dorms with highest positive sentiment
dorm_sentiment_df = pd.DataFrame(dorm_sentiment)
dorm_sentiment_df = dorm_sentiment_df.transpose()

#df in positive order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='pos', ascending=False)
display(dorm_sentiment_df)
print("Ranking based on maximum positive sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")

#df in negative order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='neg', ascending=True)
display(dorm_sentiment_df)
print("Ranking based on minimum negative sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")

#df in neutral order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='neu', ascending=False)
display(dorm_sentiment_df)
print("Ranking based on maximum neutral sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")

{'graham': {'neg': [0.08817179007239119],
  'neu': [0.5696909150411916],
  'pos': [0.3421372819778531]},
 'finn': {'neg': [0.08081351808811489],
  'neu': [0.5880970045139915],
  'pos': [0.3310894675944981]},
 'sanfilippo_flip': {'neg': [0.21430246296681854],
  'neu': [0.5735552781506589],
  'pos': [0.21214224711844795]},
 'campisi_camp': {'neg': [0.1824825368821621],
  'neu': [0.49702954434213187],
  'pos': [0.320487889505568]},
 'mcwalsh': {'neg': [0.10210787103726313],
  'neu': [0.7260305010355437],
  'pos': [0.17186159468614137]},
 'sobrato': {'neg': [0.0940956398844719],
  'neu': [0.6361769504017301],
  'pos': [0.26972745048503083]},
 'swig': {'neg': [0.24916846562275843],
  'neu': [0.5068594483675155],
  'pos': [0.24397210737245273]},
 'casa': {'neg': [0.08747939889629681],
  'neu': [0.5847758253415425],
  'pos': [0.3277447629306051]},
 'nobili': {'neg': [0.10356812737882137],
  'neu': [0.7012187114783696],
  'pos': [0.19521320291927882]}}

,neg,neu,pos
graham,[0.08817179007239119],[0.5696909150411916],[0.3421372819778531]
finn,[0.08081351808811489],[0.5880970045139915],[0.3310894675944981]
casa,[0.08747939889629681],[0.5847758253415425],[0.3277447629306051]
campisi_camp,[0.1824825368821621],[0.49702954434213187],[0.320487889505568]
sobrato,[0.0940956398844719],[0.6361769504017301],[0.26972745048503083]
swig,[0.24916846562275843],[0.5068594483675155],[0.24397210737245273]
sanfilippo_flip,[0.21430246296681854],[0.5735552781506589],[0.21214224711844795]
nobili,[0.10356812737882137],[0.7012187114783696],[0.19521320291927882]
mcwalsh,[0.10210787103726313],[0.7260305010355437],[0.17186159468614137]


Ranking based on maximum positive sentiment:
1. graham
2. finn
3. casa
4. campisi_camp
5. sobrato
6. swig
7. sanfilippo_flip
8. nobili
9. mcwalsh


,neg,neu,pos
finn,[0.08081351808811489],[0.5880970045139915],[0.3310894675944981]
casa,[0.08747939889629681],[0.5847758253415425],[0.3277447629306051]
graham,[0.08817179007239119],[0.5696909150411916],[0.3421372819778531]
sobrato,[0.0940956398844719],[0.6361769504017301],[0.26972745048503083]
mcwalsh,[0.10210787103726313],[0.7260305010355437],[0.17186159468614137]
nobili,[0.10356812737882137],[0.7012187114783696],[0.19521320291927882]
campisi_camp,[0.1824825368821621],[0.49702954434213187],[0.320487889505568]
sanfilippo_flip,[0.21430246296681854],[0.5735552781506589],[0.21214224711844795]
swig,[0.24916846562275843],[0.5068594483675155],[0.24397210737245273]


Ranking based on minimum negative sentiment:
1. finn
2. casa
3. graham
4. sobrato
5. mcwalsh
6. nobili
7. campisi_camp
8. sanfilippo_flip
9. swig


,neg,neu,pos
mcwalsh,[0.10210787103726313],[0.7260305010355437],[0.17186159468614137]
nobili,[0.10356812737882137],[0.7012187114783696],[0.19521320291927882]
sobrato,[0.0940956398844719],[0.6361769504017301],[0.26972745048503083]
finn,[0.08081351808811489],[0.5880970045139915],[0.3310894675944981]
casa,[0.08747939889629681],[0.5847758253415425],[0.3277447629306051]
sanfilippo_flip,[0.21430246296681854],[0.5735552781506589],[0.21214224711844795]
graham,[0.08817179007239119],[0.5696909150411916],[0.3421372819778531]
swig,[0.24916846562275843],[0.5068594483675155],[0.24397210737245273]
campisi_camp,[0.1824825368821621],[0.49702954434213187],[0.320487889505568]


Ranking based on maximum neutral sentiment:
1. mcwalsh
2. nobili
3. sobrato
4. finn
5. casa
6. sanfilippo_flip
7. graham
8. swig
9. campisi_camp
